# bert model

In [1]:
import os
import ast
import torch
import time
import inspect
import pandas as pd
import numpy as np
import torch.nn as nn
from torchinfo import summary
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import multiprocessing as mp
from transformers import AutoTokenizer

c:\Users\Gebruiker\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# tokenization of dataset
multiprocessing in seperate file

import os
import numpy as np
import pandas as pd
import multiprocessing as mp
from transformers import AutoTokenizer


#read data
def load():
    df = pd.read_csv('C:\\Users\\vd00r\\OneDrive\\Documenten\\GitHub\\MadManAndMachines\\pythonProject\\text_data.csv', sep=',')
    #turn string to list
    for i, id in enumerate(df['codes']):
        df['codes'][i] = eval(id)
    #drop data
    df = df.drop(['Unnamed: 0'],axis = 1)
    return(df.to_numpy())

#multiprocessing
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
cpu_c = os.cpu_count()//2 

def transform(fw):
    print(fw[0])
    tok = tokenizer.encode_plus(fw[0])
    return tok,fw[1]

def proc(fw):
    with mp.Pool(cpu_c) as pool:
        tokenized_text = []
        labels = []
        for data in pool.imap(transform,fw,chunksize=16):
            tokenized_text.append(data[0])
            labels.append(data[1])
        return tokenized_text,labels

if __name__ == '__main__':
    df = load()
    #print(df)
    a = proc(df)
    #print(a[0]['input_ids'])
    d = {"inputs": a[0],"labels": a[1]} 
    array = pd.DataFrame(data = d).to_csv('dataset.csv',index=False)
    print(array)

# Dataset class

In [3]:

df = pd.read_csv('dataset.csv', sep=',', converters={'labels': pd.eval})
print(df['labels'][0])


[10, 27]


In [4]:
class political_ads_Dataset(Dataset):
    def __init__(self):
        self.data = pd.read_csv('dataset.csv', sep=',', converters={'labels': pd.eval})
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, idx):
        input = self.data['inputs'].iloc[idx]
        input = ast.literal_eval(input)['input_ids']
        input = torch.tensor(input)
        input = F.pad(input,(0,(512-input.shape[0])),"constant",0)
        label = self.data['labels'].iloc[idx]
        return input,label[0]
    def downsize(self):
        for i, value in enumerate(self.data['labels']):
            self.data['labels'][i] = value[0]
        self.data = self.data.drop(self.data[self.data['labels'].eq(0)].sample(840).index)
        self.data = self.data.drop(self.data[self.data['labels'].eq(10)].sample(70).index)
        self.data = self.data.reset_index()
        return self.data
    def data_return(self):
        return self.data

In [5]:
data = pd.read_csv('dataset.csv', sep=',', converters={'labels': pd.eval})

for i, value in enumerate(data['labels']):
    data['labels'][i] = value[0]
    #print(value[0])
data = data.drop(data[data['labels'].eq(0)].sample(840).index)
data = data.drop(data[data['labels'].eq(10)].sample(70).index)
print(data)
count = data['labels'].value_counts()
print(count,count.index[0])
#calculate index inverse frequency

                                                 inputs labels
1     {'input_ids': [101, 24890, 8231, 25370, 16716,...     21
3     {'input_ids': [101, 2777, 112, 188, 1103, 108,...     29
5     {'input_ids': [101, 24890, 8231, 25370, 16716,...     13
6     {'input_ids': [101, 157, 2069, 25810, 2101, 11...     30
9     {'input_ids': [101, 1135, 1110, 24819, 2924, 1...     24
...                                                 ...    ...
1079  {'input_ids': [101, 3561, 119, 2103, 1107, 170...      4
1084  {'input_ids': [101, 138, 8499, 118, 17750, 346...     29
1093  {'input_ids': [101, 1409, 1809, 117, 8499, 120...     29
1108  {'input_ids': [101, 1192, 787, 1396, 7348, 115...     28
1110  {'input_ids': [101, 26385, 136, 23114, 106, 17...      6

[212 rows x 2 columns]
labels
10    25
0     23
29    20
28    15
11    14
21    13
24    13
6     11
22    11
5      9
1      9
3      8
27     7
13     7
30     6
16     5
2      5
25     3
20     2
8      2
4      2
12     1
7      1
Name: 

In [6]:
length = 422
def get_ratio(value):
    return (length-value)/length

weights = np.zeros((32,))
print(weights.shape)
for i, value in enumerate(count):
    loc = count.index[i]
    weights[loc] = get_ratio(value)

weight = torch.tensor(weights)
print(weight)

(32,)
tensor([0.9455, 0.9787, 0.9882, 0.9810, 0.9953, 0.9787, 0.9739, 0.9976, 0.9953,
        0.0000, 0.9408, 0.9668, 0.9976, 0.9834, 0.0000, 0.0000, 0.9882, 0.0000,
        0.0000, 0.0000, 0.9953, 0.9692, 0.9739, 0.0000, 0.9692, 0.9929, 0.0000,
        0.9834, 0.9645, 0.9526, 0.9858, 0.0000], dtype=torch.float64)


In [7]:
# weight matrix



# model

In [8]:
class config:
    n_embd = 768
    n_classes = 32

In [9]:
class bert_classifier(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.config = config
        self.model_bert = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased')
        self.lin = nn.Linear(config.n_embd,config.n_classes)
        #self.softmax = nn.softmax()
    def forward(self,x):
        x = self.model_bert(x)
        x = self.lin(x[1])
        return x

In [10]:
model = bert_classifier(config).to(device)

Using cache found in C:\Users\Gebruiker/.cache\torch\hub\huggingface_pytorch-transformers_main


In [11]:
summary(model,input_data=torch.ones([1,512],dtype=torch.int32).to(device))

Layer (type:depth-idx)                                       Output Shape              Param #
bert_classifier                                              [1, 32]                   --
├─BertModel: 1-1                                             [1, 768]                  --
│    └─BertEmbeddings: 2-1                                   [1, 512, 768]             --
│    │    └─Embedding: 3-1                                   [1, 512, 768]             23,440,896
│    │    └─Embedding: 3-2                                   [1, 512, 768]             1,536
│    │    └─Embedding: 3-3                                   [1, 512, 768]             393,216
│    │    └─LayerNorm: 3-4                                   [1, 512, 768]             1,536
│    │    └─Dropout: 3-5                                     [1, 512, 768]             --
│    └─BertEncoder: 2-2                                      [1, 512, 768]             --
│    │    └─ModuleList: 3-6                                  --             

# training loop

In [12]:
#-----TRAINING-HYPERPARAMETERS-----#
max_lr = 5e-6
min_lr = max_lr * 0.01
warmup_steps = 70
max_steps = 700
theta = 0.7
#-----TRAINING-HYPERPARAMETERS-----#

In [13]:
#-----BATCH-SIZE/GRADIENT-ACCUMILATION-----#
dataset = political_ads_Dataset()
weight_data = dataset.data_return()
train_set, test_set = torch.utils.data.random_split(dataset, [int(0.8*len(dataset)), len(dataset)-int(0.8*len(dataset))])
print(len(test_set))
batch_size = 64
B = 32
grad_accum_steps = batch_size // B
train_loader = DataLoader(train_set, batch_size=B, shuffle=True)
test_loader = DataLoader(test_set, batch_size=B, shuffle=True)
print(grad_accum_steps)
#-----BATCH-SIZE/GRADIENT-ACCUMILATION-----#

225
2


In [14]:
count = weight_data['labels'].value_counts()
print(count)

length = len(dataset)
def get_ratio(value):
    return (value)/length

weights = np.zeros((32,))
print(weights.shape)
for i, value in enumerate(count):
    loc = count.index[i]
    weights[loc] = min(1/(100*get_ratio(value)+1),0.7)
weight = torch.tensor(weights)
print(weight)

labels
[0]             863
[10]             74
[29]             18
[28]             14
[11]             13
               ... 
[10, 24, 29]      1
[10, 13]          1
[8, 10, 16]       1
[24, 29]          1
[10, 27, 31]      1
Name: count, Length: 63, dtype: int64
(32,)
tensor([0.0065, 0.3840, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000,
        0.0000, 0.7000, 0.7000, 0.7000, 0.7000, 0.0000, 0.0000, 0.7000, 0.0000,
        0.0000, 0.0000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.0000,
        0.7000, 0.7000, 0.7000, 0.7000, 0.7000], dtype=torch.float64)


In [15]:
for i in range(1):
    train_features, train_labels = next(iter(test_loader))
    print(train_features.shape)
    print(train_labels.shape)


torch.Size([32, 512])
torch.Size([32])


In [16]:
#-----LEARNING-RATE-SCHEDULER-----#
def get_lr(it):
    if it < warmup_steps:
        return max_lr * (it+1) / warmup_steps
    if it > max_steps:
        return min_lr
    decay = it-warmup_steps
    return max_lr - decay * (max_lr/(max_steps-warmup_steps))
#-----LEARNING-RATE-SCHEDULER-----#

In [17]:
#-----PARAMETER-SETUP-----#
for param in model.parameters():
     param.requires_grad = False
for param in model.model_bert.pooler.parameters():
     param.requires_grad = True
for param in model.lin.parameters():
     param.requires_grad = True
#-----PARAMETER-SETUP-----#

In [18]:
layer_names = []
for param in model.named_parameters():
     layer_names.append(param[0])
layer_names.reverse()
#print(layer_names)

In [19]:
#-----GRADUAL-FREEZING----#
class gradual_freezing():
    def __init__(self):
        self.unlock_ratio = 0.083
        self.i = 12
    def check_unfreeze(self,it):
        ratio = it/max_steps
        #print(ratio)
        #print(self.unlock_ratio)
        if ratio > self.unlock_ratio and self.i != 0:
            self.unlock_ratio = self.unlock_ratio +  0.083
            self.i = self.i - 1
            print(self.i)
            for param in model.model_bert.encoder.layer[self.i].parameters():
                param.requires_grad = True
                #print(param[0])
            return True
        return False
#-----GRADUAL-FREEZING-----#
a = gradual_freezing()
#for g in range(2000):
    #a.check_unfreeze(g)

In [20]:
#-----DISCRIMINATIVE-FINE-TUNING-----#
def disc_fine_tuning(lr):
    parameters = []
    list1 = []
    previous = layer_names[2].split('.')[2]
    lr_disc = lr
    for idx,name in enumerate(layer_names):
        if name.split(".")[0] == "lin":
            for p in model.named_parameters():
                if p[0] == name and p[1].requires_grad:
                    parameters.append({ "params":p[1] ,"lr":lr})
        if name.split(".")[1] == "pooler":
            for p in model.named_parameters():
                if p[0] == name and p[1].requires_grad:
                    parameters.append({ "params":p[1] ,"lr":lr_disc})
                    list1.append(p[0])
        if name.split(".")[1] == "embeddings":
            for p in model.named_parameters():
                if p[0] == name and p[1].requires_grad:
                    parameters.append({ "params":p[1] ,"lr":lr_disc})
        if name.split(".")[1] == "encoder":
                current = name.split('.')[3]
                if current != previous:
                    lr_disc = theta*lr_disc
                previous = current
                for p in model.named_parameters():
                    if p[0] == name and p[1].requires_grad:
                        parameters.append({ "params":p[1] ,"lr":lr_disc})
                        list1.append(p[0])
    return parameters, list1
#-----DISCRIMINATIVE-FINE-TUNING-----#
#parameter,list1 = disc_fine_tuning(1)
#print(parameter)
#optimizer = torch.optim.Adam(parameter)


In [21]:
#-----OPTIMIZER-----#
def configure_optimizers(model, weight_decay, learning_rate, device):
    param_dict = {pn: p for pn, p in model.named_parameters()}
    param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
    decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
    nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
    optim_groups = [
        {'params': decay_params, 'weight_decay': weight_decay},
        {'params': nodecay_params, 'weight_decay': 0.0}
    ]
    num_decay_params = sum(p.numel() for p in decay_params)
    num_nodecay_params = sum(p.numel() for p in nodecay_params) 
    print(num_decay_params,num_nodecay_params)
    fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
    use_fused = fused_available and device == "cuda"
    print(f"using fused AdamW: {use_fused}")
    optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.999), eps=1e-8, fused=use_fused)
    return optimizer  
  
optimizer = configure_optimizers(model = model, weight_decay=0.05, learning_rate=6e-4, device=device)
#-----OPTIMIZER-----#

614400 800
using fused AdamW: False


In [22]:
#-----LOSS-FUNCTION-----#
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2, ignore_index=-100, reduction='mean'):
        super().__init__()
        # use standard CE loss without reducion as basis
        self.CE = nn.CrossEntropyLoss(reduction='none', ignore_index=ignore_index)
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input, target):
        '''
        input (B, N)
        target (B)
        '''
        minus_logpt = self.CE(input, target)
        pt = torch.exp(-minus_logpt) # don't forget the minus here
        focal_loss = (1-pt)**self.gamma * minus_logpt

        # apply class weights
        if self.alpha != None:
            focal_loss *= self.alpha.gather(0, target)
        
        if self.reduction == 'mean':
            focal_loss = focal_loss.mean()
        elif self.reduction == 'sum':
            focal_loss = focal_loss.sum()
        return focal_loss

criterion = FocalLoss(alpha=weight.to(device),gamma=5)
#criterion = nn.CrossEntropyLoss()
#-----LOSS-FUNCTION-----#

In [23]:
#-----MISC-----#
checkpoint = 0
torch.set_float32_matmul_precision("high")
#-----MISC-----#

In [24]:
#model = bert_classifier(config).to(device)
for step in range(max_steps):
    last_step = (step == max_steps - 1)
    t0 = time.time()
    #optimizer.zero_grad(set_to_none=True)
    
    #validation loss
    if step % 40 == 0 or last_step:
        model.eval()
        with torch.no_grad():
            correct_acum = 0
            val_accum = 0
            val_loss_steps = 1
            for val_steps in range(val_loss_steps):
                x, y = next(iter(test_loader))
                x,y = x.to(device), y.to(device)
                with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
                    logits = model(x)
                    loss = criterion(logits,y)
                values,ind = torch.max(logits,dim = 1)
                print(y)
                print(ind)
                correct = np.sum((torch.eq(ind.to("cpu"),y.to("cpu")).numpy()))
                correct_acum = correct_acum + correct
                val_accum = val_accum + loss.detach()
            accuracy = correct_acum / (val_loss_steps * B)
            val_loss = val_accum / val_loss_steps
    
    #save checkpoint
    if step % 500 == 0:
        torch.save(model.state_dict(),f"checkpoint{checkpoint}.pt")
        checkpoint = checkpoint+1

    #minibatch process
    loss_accum = 0
    model.train()
    for mini_step in range(grad_accum_steps):
        x, y = next(iter(train_loader))
        x,y = x.to(device), y.to(device)
        with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
            logits = model(x)
            loss = criterion(logits,y)
        #print(f"loss mini_step = {loss} | ministep = {mini_step}")
        loss = loss / grad_accum_steps
        loss_accum = loss_accum + loss.detach()
        loss.backward()
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    t1 = time.time()
    dt = t1-t0
    
    a.check_unfreeze(step)
    lr = get_lr(step)
    parameter = disc_fine_tuning(lr)
    for param_group in optimizer.param_groups:
        param_group = parameter
        
    
    optimizer.step()
    print(f"| step = {step} | time = {dt:.4f}s | lr = {lr:.4e} | loss = {loss_accum.item():.6f} |norm = {norm:.4f}| latest val loss = {val_loss:.4f}| accuracy = {accuracy:.4f}")
torch.save(model.state_dict(),"Final.pt")

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([ 0,  0,  0,  0, 10,  0,  4,  2,  0, 10,  0, 27,  0, 30,  0,  0,  0,  0,
         0,  0, 21,  0,  0,  0,  0,  0,  0,  0, 11,  0,  0,  0],
       device='cuda:0')
tensor([30, 23, 30, 23, 23, 30, 23, 26, 30, 23, 30, 30, 30, 26, 30, 30, 26, 23,
        30, 30,  7, 23, 30, 30, 23, 30, 30, 23, 30, 26, 23, 30],
       device='cuda:0')
| step = 0 | time = 2.6951s | lr = 7.1429e-08 | loss = 0.569364 |norm = 1.1787| latest val loss = 0.5212| accuracy = 0.0000
| step = 1 | time = 0.3536s | lr = 1.4286e-07 | loss = 0.344757 |norm = 1.6272| latest val loss = 0.5212| accuracy = 0.0000
| step = 2 | time = 0.1885s | lr = 2.1429e-07 | loss = 0.495993 |norm = 1.0118| latest val loss = 0.5212| accuracy = 0.0000
| step = 3 | time = 0.1787s | lr = 2.8571e-07 | loss = 0.358145 |norm = 1.6058| latest val loss = 0.5212| accuracy = 0.0000
| step = 4 | time = 0.2191s | lr = 3.5714e-07 | loss = 0.398114 |norm = 1.8334| latest val loss = 0.5212| accuracy = 0.0000
| step = 5 | time = 0.1951s | lr = 4.2857e

# misc

In [ ]:
for param in model.named_parameters():
    print(param[0])

model_bert.embeddings.word_embeddings.weight
model_bert.embeddings.position_embeddings.weight
model_bert.embeddings.token_type_embeddings.weight
model_bert.embeddings.LayerNorm.weight
model_bert.embeddings.LayerNorm.bias
model_bert.encoder.layer.0.attention.self.query.weight
model_bert.encoder.layer.0.attention.self.query.bias
model_bert.encoder.layer.0.attention.self.key.weight
model_bert.encoder.layer.0.attention.self.key.bias
model_bert.encoder.layer.0.attention.self.value.weight
model_bert.encoder.layer.0.attention.self.value.bias
model_bert.encoder.layer.0.attention.output.dense.weight
model_bert.encoder.layer.0.attention.output.dense.bias
model_bert.encoder.layer.0.attention.output.LayerNorm.weight
model_bert.encoder.layer.0.attention.output.LayerNorm.bias
model_bert.encoder.layer.0.intermediate.dense.weight
model_bert.encoder.layer.0.intermediate.dense.bias
model_bert.encoder.layer.0.output.dense.weight
model_bert.encoder.layer.0.output.dense.bias
model_bert.encoder.layer.0.outp